In [1]:
import numpy as np
import pandas as pd
from __future__ import division, print_function, absolute_import

from tmm import (coh_tmm, unpolarized_RT, ellips,
                       position_resolved, find_in_structure_with_inf)

from numpy import pi, linspace, inf, array
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
%matplotlib inline
degree = pi/180

# Data Importation
While currently in static form, the dataset is collected from a dynamically appended ".dat" file. Since each row will be operated on using an imported function from the tmm module, the most ideal data structure for the storage is a dataframe.

In [8]:
# Read the raw data file, skip the first three rows without a header 
raw_data = pd.read_csv('25 nm of SiO2.dat', sep = '\t', skiprows=4 ,header=None)
#Rename the columns of the raw data frame into relevant features
raw_data.columns = ['lambda_vac', 'th_0', 'psi_exp', 'delta_exp', 'psi_exp_sigma', 'delta_exp_sigma']
raw_data

,lambda_vac,th_0,psi_exp,delta_exp,psi_exp_sigma,delta_exp_sigma
0,300.0,65.0,37.470924,108.225250,0.299714,0.653519
1,310.0,65.0,36.357937,107.988220,0.250304,0.550787
2,320.0,65.0,35.595108,109.826470,0.258704,0.580831
3,330.0,65.0,35.136696,111.131290,0.377628,0.859414
4,340.0,65.0,34.184349,112.822850,0.809246,1.887950
...,...,...,...,...,...,...
97,760.0,75.0,10.963696,78.241798,0.008063,0.044102
98,770.0,75.0,10.820502,78.041321,0.008454,0.046847
99,780.0,75.0,10.675869,77.787621,0.009062,0.050886
100,790.0,75.0,10.528711,77.541664,0.008666,0.049326


# Data Pre-processing
The angle of incidence and wavelength, i.e. *th_0* and *lambda_vac*, respectively, ared used to estimate *psi* and *delta*. The estimated parameters will be compared with the measured *psi_exp* and *delta_exp* and will then be used to calculate a mean squared error (MSE). Since the MSE is based on an estimated value for thickness, the thickness will be calculated until the MSE between the estimated and measured psi and delta are minimized.

In [3]:
#Identify the unique angles for each measurement, which will then be used for iterating over the row data
angle_list = set(raw_data['th_0'])
angle_list

{65.0, 75.0}

In [4]:
Si_nk_data = array([[300,5.084+4.247j],
[310,5.123+3.645j],
[320,5.147+3.331j],
[330,5.216+3.134j],
[340,5.336+3.008j],
[350,5.539+2.960j],
[360,6.088+2.978j],
[370,6.984+2.129j],
[380,6.676+0.888j],
[390,6.072+0.460j],
[400,5.674+0.279j],
[410,5.371+0.177j],
[420,5.132+0.124j],
[430,4.945+0.097j],
[440,4.796+0.083j],
[450,4.676+0.075j],
[460,4.576+0.068j],
[470,4.491+0.062j],
[480,4.417+0.056j],
[490,4.352+0.052j],
[500,4.294+0.048j],
[510,4.243+0.044j],
[520,4.196+0.041j],
[530,4.154+0.038j],
[540,4.116+0.035j],
[550,4.081+0.032j],
[560,4.049+0.030j],
[570,4.020+0.028j],
[580,3.993+0.026j],
[590,3.968+0.024j],
[600,3.944+0.023j],
[610,3.923+0.021j],
[620,3.902+0.020j],
[630,3.883+0.019j],
[640,3.865+0.017j],
[650,3.849+0.016j],
[660,3.833+0.015j],
[670,3.818+0.014j],
[680,3.804+0.013j],
[690,3.791+0.012j],
[700,3.778+0.011j],
[710,3.766+0.010j],
[720,3.755+0.010j],
[730,3.744+0.009j],
[740,3.734+0.008j],
[750,3.724+0.008j],
[760,3.715+0.007j],
[770,3.706+0.007j],
[780,3.698+0.006j],
[790,3.690+0.006j],
[800,3.682+0.005j],
[810,3.675+0.005j],
[820,3.668+0.004j],
[830,3.661+0.004j],
[840,3.654+0.004j],
[850,3.648+0.003j],
[860,3.642+0.003j],
[870,3.636+0.003j],
[880,3.631+0.002j],
[890,3.625+0.002j],
[900,3.620+0.002j],
[910,3.615+0.002j],
[920,3.611+0.002j],
[930,3.606+0.001j],
[940,3.602+0.001j],
[950,3.597+0.001j],
[960,3.593+0.001j],
[970,3.589+0.001j],
[980,3.585+0.001j],
[990,3.582+0.001j],
[1000,3.578+0.000j]])

In [5]:
SiO2_nk_data = array([[300,1.493],
[310,1.490],
[320,1.488],
[330,1.486],
[340,1.484],
[350,1.482],
[360,1.481],
[370,1.479],
[380,1.478],
[390,1.477],
[400,1.475],
[410,1.474],
[420,1.473],
[430,1.472],
[440,1.472],
[450,1.471],
[460,1.470],
[470,1.469],
[480,1.469],
[490,1.468],
[500,1.468],
[510,1.467],
[520,1.467],
[530,1.466],
[540,1.466],
[550,1.465],
[560,1.465],
[570,1.464],
[580,1.464],
[590,1.464],
[600,1.463],
[610,1.463],
[620,1.463],
[630,1.462],
[640,1.462],
[650,1.462],
[660,1.462],
[670,1.461],
[680,1.461],
[690,1.461],
[700,1.461],
[710,1.461],
[720,1.460],
[730,1.460],
[740,1.460],
[750,1.460],
[760,1.460],
[770,1.459],
[780,1.459],
[790,1.459],
[800,1.459],
[810,1.459],
[820,1.459],
[830,1.458],
[840,1.458],
[850,1.458],
[860,1.458],
[870,1.458],
[880,1.458],
[890,1.458],
[900,1.457],
[910,1.457],
[920,1.457],
[930,1.457],
[940,1.457],
[950,1.457],
[960,1.457],
[970,1.457],
[980,1.456],
[990,1.456],
[1000,1.456]])

In [10]:
thickness=25
d_list = [inf,thickness,inf] #in nm

#Define the wavelength range for ellipsometer data
Si_nk_fn = interp1d(Si_nk_data[:,0].real,
                          Si_nk_data[:,1], kind='quadratic')
SiO2_nk_fn = interp1d(SiO2_nk_data[:,0].real,
                          SiO2_nk_data[:,1], kind='quadratic')


for _,row in raw_data.iterrows():
            n_list = [1,SiO2_nk_fn(row['lambda_vac']),Si_nk_fn(row['lambda_vac'])]
            e_data=ellips(n_list, d_list, row['th_0']*degree, row['lambda_vac']) #in nm
            row.psi_exp = e_data['psi']/degree # angle in degrees
            row.delta_exp = (pi-e_data['Delta'])/degree # angle in degrees

In [24]:
df_list = []
plt.figure()
for angle in angle_list:
    df_list.append(raw_data[raw_data.th_0==angle])

<Figure size 432x288 with 0 Axes>

In [25]:
df_list

[    lambda_vac  th_0    psi_exp   delta_exp  psi_exp_sigma  delta_exp_sigma
 0        300.0  65.0  37.756905  108.403588       0.299714         0.653519
 1        310.0  65.0  36.542104  109.463499       0.250304         0.550787
 2        320.0  65.0  35.738298  110.632891       0.258704         0.580831
 3        330.0  65.0  35.148184  112.033743       0.377628         0.859414
 4        340.0  65.0  34.731088  113.649301       0.809246         1.887950
 5        350.0  65.0  34.537702  115.536871       0.020260         0.047908
 6        360.0  65.0  34.719282  118.495566       0.020053         0.048813
 7        370.0  65.0  34.165502  123.774649       0.019569         0.050818
 8        380.0  65.0  32.266396  127.732586       0.023662         0.066311
 9        390.0  65.0  30.558661  129.310074       0.020426         0.059979
 10       400.0  65.0  29.287871  130.464486       0.018748         0.057179
 11       410.0  65.0  28.206566  131.382958       0.017844         0.056512